## Example usage for DaskClassifier

In [1]:
import sys
import os
from pathlib import Path
DIR = Path(".").absolute()
sys.path.append(str(DIR))
os.chdir(str(DIR.parent)) # make notebook assume its in parent dir

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import json
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [3]:
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://192.168.0.104/61785/1 Dashboard: http://192.168.0.104:8787/status,Cluster Workers: 1 Cores: 8 Memory: 17.18 GB


`processes=False` is a large performance increase because threads have much faster communication when a Dask Distributed client is present. At each step, the model is copied to each worker. That's almost instant with threads because the same memory bank is shared; that's not true with processes.

`processes=False` should still be used with GPUs. There, the model/gradient calculation will live on each GPU.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from adadamp import DaskClassifier

In [6]:
# model from https://github.com/pytorch/examples/blob/master/mnist/main.py
from model import Net
client.upload_file("notebooks/model.py")

In [7]:
from torch.utils.data import Dataset
from dask.distributed import get_client

def run(
    model: nn.Module,
    train_set: Dataset,
    test_set: Dataset,
    max_epochs: int = 5,
):
    client = get_client()
    hist = []
    epochs = 0
    for epoch in range(max_epochs):
        print(f"Epoch {epoch}...", end=" ")
        model.partial_fit(train_set)
        print("done")
        model.score(test_set)  # records info in model.meta_
        datum = {"epoch": epoch + 1, **model.meta_}
        print(datum)
        hist.append(datum)
    return hist, model.get_params()

In [8]:
# transforms
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# data
train_set = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('./data', train=False, transform=transform)

In [9]:
# model
model = DaskClassifier(
    module=Net,
    weight_decay=1e-5,
    loss=nn.CrossEntropyLoss,
    optimizer=optim.SGD,
    optimizer__lr=0.01,
    optimizer__momentum=0.9,
    batch_size=128,
)


In [10]:
args = (model, train_set, test_set)
kwargs = dict(max_epochs=10)
hist, params = run(*args, **kwargs)

Epoch 0... {'n_updates': 0, 'n_data': 0, 'score__calls': 0, 'partial_fit__calls': 0, 'n_workers': 1, 'num_examples': 0, 'len_dataset': 60000}
{'n_updates': 1, 'n_data': 128, 'score__calls': 0, 'partial_fit__calls': 0, 'n_workers': 4, 'num_examples': 0, 'len_dataset': 60000, '_epochs': 0, 'damping_': 128, 'batch_size_': 128, 'lr_': 0.01}
{'n_updates': 2, 'n_data': 256, 'score__calls': 0, 'partial_fit__calls': 0, 'n_workers': 4, 'num_examples': 0, 'len_dataset': 60000, '_epochs': 0, 'damping_': 128, 'batch_size_': 128, 'lr_': 0.01}
{'n_updates': 3, 'n_data': 384, 'score__calls': 0, 'partial_fit__calls': 0, 'n_workers': 4, 'num_examples': 0, 'len_dataset': 60000, '_epochs': 0, 'damping_': 128, 'batch_size_': 128, 'lr_': 0.01}
{'n_updates': 4, 'n_data': 512, 'score__calls': 0, 'partial_fit__calls': 0, 'n_workers': 4, 'num_examples': 0, 'len_dataset': 60000, '_epochs': 0, 'damping_': 128, 'batch_size_': 128, 'lr_': 0.01}
{'n_updates': 5, 'n_data': 640, 'score__calls': 0, 'partial_fit__calls

KeyboardInterrupt: 

In [ ]:
hist[0]

In [ ]:
import pandas as pd
df = pd.DataFrame(hist)
avg_update_time = df["partial_fit__time"].sum() / df["n_updates"].max()
msg = "Avg. update time = {:0.0f}ms".format(1000 * avg_update_time)
print(msg)

In [ ]:
save = {
    'history': hist,
    'params': {k: v for k, v in params.items() if type(v) != type},
}

In [ ]:
with open('./notebooks/stats.json', 'w') as f:
    json.dump(save, f)